In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pickle

In [ ]:
df = pd.read_csv('titanic_raw.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.corr()['survived']

In [ ]:
df = df.loc[:,('pclass','age','sex','survived','embarked')]

In [ ]:
df.sample(10)

In [ ]:
df.isna().sum

In [ ]:
df.shape

In [ ]:
df = df.loc[df.embarked.notna(),:]

In [ ]:
df.nunique()

In [ ]:
df['pclass'].unique()

In [ ]:
df['embarked'].unique()

In [ ]:
df_x = df.drop('survived', axis='columns')
df_y = df.survived

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size= 0.3, stratify = df_y)

In [ ]:
num_pipeline = Pipeline([
('imputer', SimpleImputer(strategy='median')),
('std_scaler', StandardScaler()),
])

In [ ]:
col_transformer = ColumnTransformer([
    ('num',num_pipeline, ['age','pclass',]),
    ('cat', OneHotEncoder(), ['sex', 'embarked'])
])

In [ ]:
pipeline = Pipeline([
('ct', col_transformer),
('clf', LogisticRegression())
])


In [ ]:
pipeline.fit(x_train, y_train)

In [ ]:
scores = cross_val_score(pipeline, x_train, y_train, scoring='accuracy', cv=5)

In [ ]:
scores.mean()

In [ ]:
pickle.dump(pipeline, open('pipeline.pkl', 'wb'))

In [ ]:
y_test_predicted = pipeline.predict(x_test)

In [ ]:
accuracy_score(y_test, y_test_predicted)

In [ ]:
cm = confusion_matrix(y_test, y_test_predicted, labels=pipeline.classes_)
cm

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipeline.classes_)


In [ ]:
disp.plot()
plt.show()


In [ ]:
pipeline = pickle.load(open('pipeline.pkl', 'rb'))